# Install Library

In [ ]:
# Install necessary library
!pip install langchain-core langgraph>0.2.27
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.5/411.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.0 MB/s eta 0:00:00


# Import Library

In [ ]:
# Utilization
import os
from rich.panel import Panel
from rich.syntax import Syntax
from rich.console import Console
from google.colab import userdata
from IPython.display import Image, display
from typing_extensions import Annotated, TypedDict
from typing import Sequence, Literal, Optional, Dict, Any
# LangChain
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import AIMessage
from langchain_core.messages import BaseMessage
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.language_models.chat_models import BaseChatModel
from langchain.callbacks.tracers.langchain import wait_for_all_tracers
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# LangGraph
from langgraph.types import Command
from langgraph.graph.message import add_messages
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph
# LangSmith
from langsmith import Client

# How to set up key in colab
If you use colab you can use private key of google, on the left side of navigation bar, I can see a key -> just simple set key is name and next blank is value \
Function: `userdata("KEY_NAME")`

# How to set up key in local
you can create file .env and use load_dotenv() to load environment file \
Function: `load_dotenv()`

# LangSmith Setup

In [ ]:
# Extract from private key for LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = userdata.get('LANGCHAIN_PROJECT')
os.environ["LANGCHAIN_ENDPOINT"] = userdata.get('LANGCHAIN_ENDPOINT')
client = Client()

# LLMs Connection

In [ ]:
# Extract from private key for Azure OpenAI
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_API_VERSION"] = userdata.get('AZURE_OPENAI_API_VERSION')
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = userdata.get('AZURE_OPENAI_DEPLOYMENT_NAME')

# Connect Azure OpenAI
# you can change type of model in langchain website: https://python.langchain.com/docs/tutorials/chatbot/
model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    model_name=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
)

# Define template for system prompt

In [ ]:
# Define System prompt instruction for sales
# example for you, you can change it
def system_prompt_instructions_sales() -> str:
    return (
'''
# **Role Description:**

- You are a professional copywriter and sales strategist with expertise in creating captivating livestream sales scripts tailored to diverse product categories and brands.
- Your task is to develop a comprehensive, engaging script for a livestream sales session on e-commerce platforms, specifically for [Brand Name], a premium [Product Type] brand.
- Craft a narrative that not only showcases the product's unique value but also creates an emotional connection with the audience, ultimately driving viewer engagement and sales conversions.
- Leverage storytelling techniques, cultural insights, and strategic communication to transform viewers into potential customers.
.... More
'''
)

# Chatbot with multi-user and multi-session

In [ ]:
# Create config
# if you want to many users has history for them
# each user has many sessions
config = {"configurable": {"user_id": "123", "thread_id": "abc123"}}

# Define Template
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt_instructions_sales(),
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
# Sample Workflow
def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Chat history
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
# promot the first time
# Example
query = '''
Hãy viết kịch bản với sản phẩm dưới đây, độ dài mỗi mục tối thiểu từ 300 đến 400 chữ. Đây là thông tin sản phẩm:

Sản phẩm 1:

Tên sản phẩm: Nước tăng lực Pepsi
'''

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
print(output["messages"][-1])
output["messages"][-1].pretty_print()

content='## **Tên sản phẩm**: Nước tăng lực Pepsi\n\n### **Mở đầu sản phẩm hoặc chuyển tiếp sản phẩm khác**:\n"Các anh chị em ơi, hôm nay chúng ta đã trải qua rất nhiều sản phẩm tuyệt vời, nhưng giờ em muốn giới thiệu đến mọi người một sản phẩm không thể thiếu trong cuộc sống hàng ngày của chúng ta, đó chính là Nước tăng lực Pepsi! Nước tăng lực này không chỉ cung cấp năng lượng mà còn giúp chúng ta tỉnh táo hơn trong những giờ phút căng thẳng. Hãy cùng em khám phá điều gì đặc biệt ở sản phẩm này nhé!"\n\n### **Giới thiệu sản phẩm**:\n"Nước tăng lực Pepsi là một sản phẩm tuyệt vời dành cho tất cả mọi người từ trẻ em trên 1 tuổi cho đến người lớn. Trong cuộc sống bận rộn hiện đại, đôi khi chúng ta cần một chút hỗ trợ để giữ cho cơ thể hoạt động hiệu quả và đầy năng lượng. Nước tăng lực Pepsi ra đời để đáp ứng nhu cầu đó. Với mỗi ngụm nước, bạn sẽ cảm nhận được sự tươi mới và sức mạnh tràn đầy. Hãy tưởng tượng bạn đang làm việc chăm chỉ hay tham gia một hoạt động thể thao và chỉ cần một 

In [ ]:
# Enhance performance

query = "Viết lại văn bản theo văn phong vui vẻ"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

## **Tên sản phẩm**: Laptop Siêu Mạnh

### **Mở đầu sản phẩm hoặc chuyển tiếp sản phẩm khác**:
Chào mừng cả nhà đến với livestream hôm nay! Hôm nay, chúng ta sẽ cùng nhau khám phá một sản phẩm cực kỳ thú vị và không thể thiếu trong cuộc sống hiện đại - đó chính là Laptop Siêu Mạnh! Nếu bạn đang tìm kiếm một người bạn đồng hành đáng tin cậy cho công việc, học tập hoặc giải trí, thì đây chính là lựa chọn hoàn hảo dành cho bạn đấy!

### **Giới thiệu sản phẩm**:
Laptop Siêu Mạnh không chỉ đơn thuần là một chiếc máy tính, mà còn là một công cụ hỗ trợ tuyệt vời giúp bạn thực hiện mọi ý tưởng và dự án của mình. Với thiết kế tinh tế và hiệu suất ấn tượng, sản phẩm này sẽ khiến bạn cảm thấy như đang sở hữu một siêu anh hùng trong tay! Hãy thử tưởng tượng xem, bạn có thể hoàn thành công việc một cách nhanh chóng, hay thậm chí là chơi game thỏa thích mà không lo lag một tí nào. Điều đó thật tuyệt vời phải không nào?

In [ ]:
# Pre-format for output
class PrettyChatbot:
    def __init__(self, model: BaseChatModel, config):
        """
        Initialize the chatbot with a configurable language model.

        :param model: The specific Claude model to use
        """
        # Initialize console for pretty printing
        self.console = Console()

        # Set up the chat model
        self.chat_model = model
        self.config = config

    def display_welcome_banner(self):
        """Display an attractive welcome banner for the chatbot."""
        welcome_text = Panel.fit(
            "[bold cyan]Welcome to PrettyChatbot[/bold cyan]\n"
            "[green]Type 'FINISH' to exit[/green]",
            border_style="bold blue"
        )
        self.console.print(welcome_text)

    def get_user_input(self) -> str:
        """
        Prompt for and return user input with rich formatting.

        :return: User's input query
        """
        return self.console.input(
            Panel(
                "[bold green]Enter your query:[/bold green]",
                border_style="green"
            )
        )

    def process_query(self, query: str) -> str:
        """
        Process the user's query and return the AI's response.

        :param query: User's input query
        :return: AI's response
        """
        try:
            # Create message input
            input_messages = [HumanMessage(content=query)]

            # Invoke the chat model
            response = self.chat_model.invoke({"messages": input_messages}, self.config)
            return response["messages"][-1].content
        except Exception as e:
            return f"[Error] {str(e)}"

    def display_response(self, response: str):
        """
        Display the AI's response with rich formatting.

        :param response: AI's response text
        """
        response_panel = Panel(
            response,
            title="[bold blue]Chatbot Response[/bold blue]",
            border_style="cyan"
        )
        self.console.print(response_panel)

In [ ]:
# Chatbot
class Chatbot:
    def __init__(self, model: BaseChatModel, config: Dict):
        # simple model
        self.model = model
        # advanced model
        self.config = config
        self.main_model = self.workflow()
        self.chat_console_instance = PrettyChatbot(self.main_model, self.config)

    def _call_model(self, state: MessagesState):
        prompt_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt_instructions_sales(),
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
        )
        prompt = prompt_template.invoke(state)
        response = self.model.invoke(prompt)
        return {"messages": response}

    def workflow(self):
        # Define workflow
        workflow = StateGraph(state_schema=MessagesState)
        workflow.add_edge(START, "model")
        workflow.add_node("model", self._call_model)

        # Chat history
        memory = MemorySaver()
        main_model = workflow.compile(checkpointer=memory)
        return main_model

    def prompt_first_product(self):
        prompt = 'Hãy viết kịch bản với sản phẩm dưới đây, độ dài mỗi mục tối thiểu từ 300 đến 400 chữ. Đây là thông tin sản phẩm:'
        first_product = '''
        Sản phẩm 1:

        Tên sản phẩm: Nước tăng lực Pepsi
        '''
        return prompt + first_product

    def sample_first_product(self):
        input_messages = [HumanMessage(self.prompt_first_product())]
        output = self.main_model.invoke({"messages": input_messages}, self.config)
        return output["messages"][-1].content


    def chat_console_run(self):
        self.chat_console_instance.display_welcome_banner()
        self.chat_console_instance.display_response(self.sample_first_product())
        while True:
            # Get user input
            user_input = self.chat_console_instance.get_user_input()

            # Check for exit condition
            if user_input.upper() == "FINISH":
                break

            # Process query and get response
            response = self.chat_console_instance.process_query(user_input)

            # Display response
            self.chat_console_instance.display_response(response)

        # Farewell message
        goodbye_panel = Panel.fit(
            "[bold magenta]Thank you for chatting! Goodbye.[/bold magenta]",
            border_style="bold red"
        )
        self.chat_console_instance.console.print(goodbye_panel)

In [ ]:
config = {"configurable": {"user_id": "124", "thread_id": "abc124"}}
chat = Chatbot(model, config)
chat.chat_console_run()

╭──────────────────────────╮
│ Welcome to PrettyChatbot │
│ Type 'FINISH' to exit    │
╰──────────────────────────╯

╭─────────────────────────────────────────────── Chatbot Response ────────────────────────────────────────────────╮
│ ## **Tên sản phẩm**: Nước tăng lực Pepsi                                                                        │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ## **Mở đầu sản phẩm hoặc chuyển tiếp sản phẩm khác**:                                                          │
│                                                                                                                 │
│ "Cả nhà mình ơi, hôm nay em muốn chia sẻ với mọi người một sản phẩm tuyệt vời mà chắc chắn sẽ giúp chúng ta     │
│ thêm năng lượng và sự tỉnh táo trong những ngày bận rộn này. Đó chính là Nước tăng lực Pepsi! Không chỉ là một  │
│ món đồ uống thông thường, mà đây là một nguồn sức mạnh giúp chúng ta vượt qua mọi thử thách. Hãy cùng em khám   │
│ phá sản phẩm này nhé!"                                                                                          │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ## **Giới thiệu sản phẩm**:                                                                                     │
│                                                                                                                 │
│ "Nước tăng lực Pepsi không chỉ đơn thuần là một thức uống mà nó là một bước tiến trong việc cung cấp năng lượng │
│ cho cả gia đình mình. Được thiết kế đặc biệt để tiếp thêm sức mạnh và giúp chúng ta luôn tỉnh táo, sản phẩm này │
│ hoàn toàn phù hợp cho mọi lứa tuổi, từ trẻ em trên 1 tuổi cho đến người lớn. Trong một thế giới đầy căng thẳng  │
│ và áp lực, việc tìm kiếm một nguồn năng lượng nhanh chóng là điều rất cần thiết. Chính vì vậy, Nước tăng lực    │
│ Pepsi ra đời như một giải pháp tuyệt vời. Hãy thử tưởng tượng bạn đang làm việc hay học tập, và đột nhiên cảm   │
│ thấy mệt mỏi, không thể tập trung. Lúc này, một lon Nước tăng lực Pepsi sẽ giúp bạn tỉnh táo và tràn đầy năng   │
│ lượng, giúp bạn hoàn thành công việc một cách xuất sắc."                                                        │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ## **Giữ chân người xem**:                                                                                      │
│                                                                                                                 │
│ "Các anh chị em có bao giờ cảm thấy mệt mỏi giữa giờ học hay giờ làm không? Em chắc chắn rằng ai trong chúng ta │
│ cũng đã từng trải qua cảm giác đó. Vậy tại sao không thử ngay Nước tăng lực Pepsi để giúp mình tỉnh táo hơn?    │
│ Hãy chia sẻ với em trong chat nhé, mọi người thường làm gì để lấy lại năng lượng trong những lúc như vậy? Đừng  │
│ ngần ngại, em rất muốn nghe ý kiến của cả nhà mình!"                                                            │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                       

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Enter your query:                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Hãy viết lại văn bản với văn phong vui vẻ


╭─────────────────────────────────────────────── Chatbot Response ────────────────────────────────────────────────╮
│ ## **Tên sản phẩm**: Nước tăng lực Pepsi                                                                        │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ## **Mở đầu sản phẩm hoặc chuyển tiếp sản phẩm khác**:                                                          │
│                                                                                                                 │
│ "Cả nhà ơi, hôm nay em muốn giới thiệu cho mọi người một sản phẩm vô cùng tuyệt vời mà chắc chắn sẽ làm bừng    │
│ sáng cả ngày của chúng ta! Đó chính là Nước tăng lực Pepsi! Không chỉ là một món đồ uống thông thường, mà đây   │
│ là bí quyết giúp chúng ta tràn đầy năng lượng và luôn tỉnh táo. Nghe có hấp dẫn không nào? Cùng em khám phá sản │
│ phẩm này nhé!"                                                                                                  │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ## **Giới thiệu sản phẩm**:                                                                                     │
│                                                                                                                 │
│ "Nước tăng lực Pepsi là một người bạn đồng hành tuyệt vời trong cuộc sống bận rộn của chúng ta. Sản phẩm này    │
│ không chỉ đơn thuần là một thức uống mà còn là nguồn sức mạnh giúp mọi thành viên trong gia đình đều có thể     │
│ thêm năng lượng để vượt qua mọi thử thách. Dù bạn là học sinh đang ôn thi hay nhân viên văn phòng cần thêm động │
│ lực để hoàn thành công việc, Nước tăng lực Pepsi sẽ là người bạn lý tưởng!                                      │
│                                                                                                                 │
│ Hãy tưởng tượng nhé: bạn đang làm việc chăm chỉ, nhưng bỗng dưng cảm thấy mệt mỏi và không thể tập trung. Thế   │
│ thì sao không thử ngay một lon Nước tăng lực Pepsi? Một chút vị chua ngọt, sảng khoái sẽ giúp bạn tỉnh táo ngay │
│ lập tức! Và đừng quên, sản phẩm này hoàn toàn an toàn cho trẻ em từ 1 tuổi trở lên, nên cả gia đình mình đều có │
│ thể thưởng thức!"                                                                                               │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ## **Giữ chân người xem**:                                                                                      │
│                                                                                                                 │
│ "Các anh chị em ơi, có bao giờ mọi người cảm thấy mệt mỏi giữa giờ học hay giờ làm không? Em tin chắc rằng ai   │
│ trong chúng ta cũng đã từng trải qua cảm giác đó rồi! Vậy thì, mọi người thường làm gì để lấy lại năng lượng    │
│ trong những lúc ấy? Hãy chia sẻ với em trong chat nhé! Em rất hào hứng muốn nghe những bí quyết của cả nhà      │
│ mình!"                                                                                                          │
│                                                       

...

Hãy viết lại với mỗi đoạn 50 từ 


╭─────────────────────────────────────────────── Chatbot Response ────────────────────────────────────────────────╮
│ ## **Tên sản phẩm**: Nước tăng lực Pepsi                                                                        │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ## **Mở đầu sản phẩm hoặc chuyển tiếp sản phẩm khác**:                                                          │
│                                                                                                                 │
│ "Cả nhà ơi, hôm nay em muốn giới thiệu cho mọi người một sản phẩm cực kỳ tuyệt vời giúp chúng ta tràn đầy năng  │
│ lượng, đó chính là Nước tăng lực Pepsi! Cùng em khám phá bí quyết này nhé!"                                     │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ## **Giới thiệu sản phẩm**:                                                                                     │
│                                                                                                                 │
│ "Nước tăng lực Pepsi không chỉ là một món đồ uống mà còn là nguồn sức mạnh giúp mọi thành viên trong gia đình   │
│ tràn đầy năng lượng. Dù bạn học tập hay làm việc, sản phẩm này sẽ là người bạn lý tưởng để vượt qua mọi thử     │
│ thách!"                                                                                                         │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ## **Giữ chân người xem**:                                                                                      │
│                                                                                                                 │
│ "Các anh chị em ơi, có bao giờ cảm thấy mệt mỏi giữa giờ học hay giờ làm không? Hãy chia sẻ với em trong chat   │
│ nhé! Em rất muốn biết mọi người thường làm gì để lấy lại năng lượng trong những lúc như vậy!"                   │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ## **Đặc điểm sản phẩm**:                                                                                       │
│                                                                                                                 │
│ "Nước tăng lực Pepsi nổi bật với khả năng tăng cường sự tỉnh táo. Mỗi sip đều mang đến hương vị thơm ngon, giúp │
│ bạn luôn cảm thấy sảng khoái. Sản phẩm này thật tiện lợi khi mang theo bên mình trong nhiều tình huống khác     │
│ nhau!"                                                                                                          │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                       

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Enter your query:                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

FINISH


╭──────────────────────────────────╮
│ Thank you for chatting! Goodbye. │
╰──────────────────────────────────╯

# END